In [43]:
# KUBECONFIG
# contexts:
# - context:
#     cluster: hdha-cluster
#     namespace: kubeflow
#     user: hdha-cluster
#   name: hdha-cluster
# current-context: hdha-cluster

In [2]:
#no ingress requires the port forward
#!kubectl port-forward -n kubeflow svc/ml-pipeline-ui 8080:80

Forwarding from 127.0.0.1:8080 -> 3000
Forwarding from [::1]:8080 -> 3000
^C


In [9]:
import kfp
import kfp.components as comp
from typing import NamedTuple
import numpy as np
import kfp.dsl as dsl


# requires port forward. 
CLUSTER = 'http://localhost:8080'

# CLUSTER = 'http://147.251.253.24'
client = kfp.Client(CLUSTER)
client.list_pipelines().pipelines[-1]

{'created_at': datetime.datetime(2021, 6, 28, 16, 28, 41, tzinfo=tzutc()),
 'default_version': {'code_source_url': None,
                     'created_at': datetime.datetime(2021, 6, 28, 16, 28, 41, tzinfo=tzutc()),
                     'id': '188ea1d9-e422-4da5-8381-a907735e5952',
                     'name': '[Tutorial] DSL - Control structures',
                     'package_url': None,
                     'parameters': None,
                     'resource_references': [{'key': {'id': '188ea1d9-e422-4da5-8381-a907735e5952',
                                                      'type': 'PIPELINE'},
                                              'name': None,
                                              'relationship': 'OWNER'}]},
 'description': '[source '
                'code](https://github.com/kubeflow/pipelines/tree/2b5a5dd2d0beea39dd2d961d31ebad2ad363742a/samples/tutorials/DSL%20-%20Control%20structures) '
                'Shows how to use conditional execution and exit handle

In [14]:
def simple_echo(i: int) -> int:
 return i

echo_op = comp.func_to_container_op(simple_echo)

In [17]:
# strongly typed
echo_op(1)

TaskSpec(component_ref=ComponentReference(name=None, digest=None, tag=None, url=None, spec=ComponentSpec(name='Simple echo', description=None, metadata=None, inputs=[InputSpec(name='i', type='Integer', description=None, default=None, optional=False, annotations=None)], outputs=[OutputSpec(name='Output', type='Integer', description=None, annotations=None)], implementation=ContainerImplementation(container=ContainerSpec(image='python:3.7', command=['sh', '-ec', 'program_path=$(mktemp)\nprintf "%s" "$0" > "$program_path"\npython3 -u "$program_path" "$@"\n', 'def simple_echo(i):\n return i\n\ndef _serialize_int(int_value: int) -> str:\n    if isinstance(int_value, str):\n        return int_value\n    if not isinstance(int_value, int):\n        raise TypeError(\'Value "{}" has type "{}" instead of int.\'.format(str(int_value), str(type(int_value))))\n    return str(int_value)\n\nimport argparse\n_parser = argparse.ArgumentParser(prog=\'Simple echo\', description=\'\')\n_parser.add_argument(

In [19]:
@dsl.pipeline(name='echo pipe',description='echo')
def echo_pipe(x='1'):
    echo_task = echo_op(x)

In [21]:
arguments = {'x': '7'}
client.create_run_from_pipeline_func(echo_pipe, arguments=arguments)

/home/m/anaconda3/envs/kf38/lib/python3.8/site-packages/kfp/compiler/compiler.py:1149: UserWarning: Cannot validate the compiled workflow. Found the argo program in PATH, but it's not usable. argo v2.4.3 should work.
  warnings.warn("Cannot validate the compiled workflow. Found the argo program in PATH, but it's not usable. argo v2.4.3 should work.")


RunPipelineResult(run_id=6724d4e5-b014-42da-80e5-42c2539b3dd3)

In [20]:
def add(a: float, b: float) -> float:
 '''Calculates sum of two arguments'''
 return a + b

add_op = comp.func_to_container_op(add)

In [27]:
from typing import NamedTuple

'''Divides two numbers and calculate the quotient and remainder'''
def my_divmod(dividend: float, divisor:float) -> NamedTuple('MyDivmodOutput', [('quotient', float), ('remainder', float)]):
    # imports must be
    # done within the function. That is because global imports from the notebook will not
    # be packaged into the containers we create.
    import numpy as np
    def divmod_helper(dividend, divisor):
        return np.divmod(dividend, divisor)
    
    (quotient, remainder) = divmod_helper(dividend, divisor)
 
    from collections import namedtuple
    divmod_output = namedtuple('MyDivmodOutput', ['quotient', 'remainder'])
    return divmod_output(quotient, remainder)

In [28]:
divmod_op = comp.func_to_container_op(my_divmod, base_image='tensorflow/tensorflow:1.14.0-py3')

In [29]:
divmod_op

<function My divmod(dividend: float, divisor: float)>

In [32]:
@dsl.pipeline(
 name='Calculation pipeline',
 description='A toy pipeline that performs arithmetic calculations.'
)
def calc_pipeline(
 a='a',
 b='7',
 c='17',
):
    #Passing pipeline parameter and a constant value as operation arguments
    add_task = add_op(a, 4) #Returns a dsl.ContainerOp class instance.
    #Passing a task output reference as operation arguments
    #For an operation with a single return value, the output
    # reference can be accessed using `task.output`
    # or `task.outputs['output_name']` syntax
    divmod_task = divmod_op(add_task.output, b)
    #For an operation with multiple return values, the output references
    # can be accessed using `task.outputs['output_name']` syntax
    result_task = add_op(divmod_task.outputs['quotient'], c) 

In [35]:
#IMPORTANT, no trailing slash, cuz not found!
client = kfp.Client("http://147.251.253.24")
#Specify pipeline argument values
# arguments = {'a': '7', 'b': '8'} #whatever makes sense for new version
#Submit a pipeline run
# client.create_run_from_pipeline_func(calc_pipeline, arguments=arguments)

In [36]:
arguments = {'a': '7', 'b': '8'}
client.create_run_from_pipeline_func(calc_pipeline, arguments=arguments)

RunPipelineResult(run_id=115f5b32-6c08-4d06-9aac-e5c06d1d5092)

In [ ]:
# It’s also worth noting that adding the @kfp.dsl.component
# annotation instructs the Kubeflow compiler to turn on static type checking:

In [37]:
dvop = dsl.VolumeOp(name="create_pvc", resource_name="my-pvc-2", size="5Gi", modes=dsl.VOLUME_MODE_RWO)

In [38]:
fetch = kfp.dsl.ContainerOp(name='download',
 image='busybox',
 command=['sh', '-c'],
 arguments=[
 'sleep 1;'
 'mkdir -p /tmp/data;'
 'wget ' + data_url +
 ' -O /tmp/data/results.csv'
 ],
 file_outputs={'downloaded': '/tmp/data'})
 # This expects a directory of inputs not just a single file

NameError: name 'data_url' is not defined

In [40]:
client = kfp.Client(CLUSTER)

In [42]:
client.get_experiment(experiment_name = "football")

{'created_at': datetime.datetime(2021, 6, 28, 15, 24, 2, tzinfo=tzutc()),
 'description': 'football experiment\n',
 'id': '16b7cebb-3082-49b9-a685-4b9d3b73bf55',
 'name': 'football',
 'resource_references': None,
 'storage_state': 'STORAGESTATE_AVAILABLE'}